In [ ]:
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm
from scipy import ndimage
from pathlib import Path
import argparse

def calculate_3d_lbp(image, radius=1, num_points=26):
    """
    Calculate 3D Local Binary Pattern for a 3D image.
    
    Parameters:
    -----------
    image : 3D numpy array
        The input 3D image
    radius : int, optional
        The radius of the circle (default is 1)
    num_points : int, optional
        Number of sampling points (default is 26 for 3D)
        
    Returns:
    --------
    lbp_histogram : 1D numpy array
        Histogram of LBP values
    """
    # Padding the image to handle border cases
    padded_image = np.pad(image, radius, mode='constant', constant_values=0)
    
    # Get image dimensions
    depth, height, width = image.shape
    
    # Initialize LBP array
    lbp_image = np.zeros_like(image, dtype=np.uint32)
    
    # Generate 3D neighborhood indices
    x, y, z = np.meshgrid(np.arange(-radius, radius + 1),
                          np.arange(-radius, radius + 1),
                          np.arange(-radius, radius + 1))
    
    # Exclude the center point and select only the points within the sphere
    mask = (x**2 + y**2 + z**2 <= radius**2) & (x**2 + y**2 + z**2 > 0)
    
    # Filter coordinates based on mask
    x_filtered = x[mask]
    y_filtered = y[mask]
    z_filtered = z[mask]
    
    # Limit the number of points if necessary
    if len(x_filtered) > num_points:
        indices = np.random.choice(len(x_filtered), num_points, replace=False)
        x_filtered = x_filtered[indices]
        y_filtered = y_filtered[indices]
        z_filtered = z_filtered[indices]
    
    # Calculate LBP
    powers = 2 ** np.arange(len(x_filtered), dtype=np.uint32)
    
    for i in range(depth):
        for j in range(height):
            for k in range(width):
                center_value = padded_image[i + radius, j + radius, k + radius]
                lbp_code = 0
                
                for idx, (dx, dy, dz) in enumerate(zip(x_filtered, y_filtered, z_filtered)):
                    neighbor_value = padded_image[i + radius + dz, j + radius + dy, k + radius + dx]
                    if neighbor_value >= center_value:
                        lbp_code += powers[idx]
                
                lbp_image[i, j, k] = lbp_code
    
    # Create histogram
    max_bins = 2**len(x_filtered)
    histogram = np.histogram(lbp_image, bins=np.arange(0, max_bins + 1))[0]
    
    # Normalize histogram
    if np.sum(histogram) > 0:
        histogram = histogram.astype(float) / np.sum(histogram)
        
    return histogram

def extract_3d_lbp_from_folders(dataset_path, class0_folder, class1_folder, output_file, radius=1, num_points=26):
    """
    Extract 3D LBP features from .nii.gz files in two class folders and save to a text file.
    
    Parameters:
    -----------
    dataset_path : str
        Path to the dataset folder
    class0_folder : str
        Name of the folder corresponding to class 0
    class1_folder : str
        Name of the folder corresponding to class 1
    output_file : str
        Path to save the output text file
    radius : int, optional
        The radius for LBP calculation
    num_points : int, optional
        Number of sampling points for LBP calculation
    """
    # Ensure the dataset path exists
    if not os.path.exists(dataset_path):
        raise ValueError(f"Dataset path {dataset_path} does not exist.")
    
    # Create full paths to class folders
    class0_path = os.path.join(dataset_path, class0_folder)
    class1_path = os.path.join(dataset_path, class1_folder)
    
    # Ensure class folders exist
    if not os.path.exists(class0_path):
        raise ValueError(f"Class 0 folder {class0_path} does not exist.")
    if not os.path.exists(class1_path):
        raise ValueError(f"Class 1 folder {class1_path} does not exist.")
    
    # Initialize list to store results
    results = []
    
    # Process class 0 images
    class0_files = [f for f in os.listdir(class0_path) if f.endswith('.nii.gz')]
    print(f"Processing {len(class0_files)} files from class 0...")
    
    for file_name in tqdm(class0_files):
        file_path = os.path.join(class0_path, file_name)
        try:
            # Load the image
            nii_img = nib.load(file_path)
            img_data = nii_img.get_fdata()
            
            # Calculate LBP histogram
            histogram = calculate_3d_lbp(img_data, radius, num_points)
            
            # Store the result
            results.append((0, histogram))
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Process class 1 images
    class1_files = [f for f in os.listdir(class1_path) if f.endswith('.nii.gz')]
    print(f"Processing {len(class1_files)} files from class 1...")
    
    for file_name in tqdm(class1_files):
        file_path = os.path.join(class1_path, file_name)
        try:
            # Load the image
            nii_img = nib.load(file_path)
            img_data = nii_img.get_fdata()
            
            # Calculate LBP histogram
            histogram = calculate_3d_lbp(img_data, radius, num_points)
            
            # Store the result
            results.append((1, histogram))
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Save results to text file
    print(f"Saving results to {output_file}...")
    with open(output_file, 'w') as f:
        for class_label, histogram in results:
            histogram_str = ','.join(map(str, histogram))
            f.write(f"{class_label}, {histogram_str}\n")
    
    print(f"Processing complete. Results saved to {output_file}")
    print(f"Processed {len(class0_files)} class 0 images and {len(class1_files)} class 1 images.")

In [ ]:
dataset = r""

extract_3d_lbp_from_folders(
    args.dataset,
    args.class0,
    args.class1,
    args.output,
    args.radius,
    args.points
)